<a href="https://colab.research.google.com/github/JelenaMitrovic/Jelena/blob/master/reddish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# @title Download Data
from google.colab import drive, auth
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [0]:
# @title Preparation
!pip install -q keras-bert keras-rectified-adam
#!pip install -q keras-bert-tpu keras-rectified-adam
!pip install tensorflow==1.5

import os
from random import seed
import numpy as np

from keras_bert.datasets import get_pretrained, PretrainedList
#model_path = get_pretrained(PretrainedList.flaubert_base_uncased)
print (PretrainedList.__dict__)

SEQ_LEN = 100
BATCH_SIZE = 8
EPOCHS = 5
LR = 1e-5


bert_model = {
        'model': 'uncased_L-12_H-768_A-12',
        'config': 'bert_config.json',
        'checkpoint': 'bert_model.ckpt',
        'vocab': 'vocab.txt'
}    

data_path = '/content/drive/My Drive/Colab Notebooks/bert-models/'
pretrained_path = os.path.join(data_path, bert_model['model'])
config_path = os.path.join(pretrained_path, bert_model['config'])
checkpoint_path = os.path.join(pretrained_path, bert_model['checkpoint'])
vocab_path = os.path.join(pretrained_path, bert_model['vocab'])


     |████████████████████████████████| 44.4MB 74kB/s 
     |████████████████████████████████| 3.0MB 15.0MB/s 
     |████████████████████████████████| 890kB 42.8MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107220 sha256=0db033903b28aa94768f3fd9e1d3ffe1a2a71a05d2ad591cf1e024347ae04cbe
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
Successfully built html5lib
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installation: bleach 3.1.3
    Uninstalling bleach-3.1.3:
      Successfully uninstalled bleach-3.1.3
  Found existing installation: tensorflow 2.2.0rc1
    Uninstalling tensorflow-2.2.0rc1:
      Successfully uninstalled tensorflow-2.2.0rc1


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:493: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:494: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:495: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:496: FutureWarn

{'__module__': 'keras_bert.datasets.pretrained', '__test__': PretrainedInfo(url='https://github.com/CyberZHG/keras-bert/archive/master.zip', extract_name='keras-bert-master', target_name='keras-bert'), 'multi_cased_base': 'https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip', 'chinese_base': 'https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip', 'wwm_uncased_large': 'https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip', 'wwm_cased_large': 'https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip', 'chinese_wwm_base': PretrainedInfo(url='https://storage.googleapis.com/hfl-rc/chinese-bert/chinese_wwm_L-12_H-768_A-12.zip', extract_name='publish', target_name='chinese_wwm_L-12_H-768_A-12'), '__dict__': <attribute '__dict__' of 'PretrainedList' objects>, '__weakref__': <attribute '__weakref__' of 'PretrainedList' objects>, '__doc__': None}


In [0]:
# @title Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint

token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,
)

In [0]:
import os
import numpy as np
from tqdm import tqdm
from keras_bert import Tokenizer
import pickle
import csv

tokenizer = Tokenizer(token_dict)

def load_data(path, delimiter=",", fields=['id', 'text', 'label'], transform={}):
    global tokenizer
    indices, sentiments = [], []
    with open(path, 'r') as f:
        reader = csv.DictReader(f, delimiter=delimiter)
        for row in reader:
            ids, segments = tokenizer.encode(row[fields[1]], max_len=SEQ_LEN)
            indices.append(ids)
            sentiments.append(transform[row[fields[2]]])

    items = list(zip(indices, sentiments))
    np.random.shuffle(items)
    indices, sentiments = zip(*items)
    indices = np.array(indices)
    mod = indices.shape[0] % BATCH_SIZE
    if mod > 0:
        indices, sentiments = indices[:-mod], sentiments[:-mod]
    return [indices, np.zeros_like(indices)], np.array(sentiments)
  
base_path = '/content/drive/My Drive/Colab Notebooks/data/'

# below:
# transform={"NOTABU":0, "IMP":1, "EXP":1} for binary abusive classification
# transform={"NOTABU":0, "IMP":1, "EXP":2} for three-way classification

# use this for the baseline
#train_path = os.path.join(base_path, 'abuseval/train.csv')
#train_x, train_y = load_data(train_path, delimiter="\t", fields=['id', 'text', 'abuse'], transform={"NOTABU":0, "IMP":1, "EXP":2}) 

# use this for the distant supervision data only
#train_path = os.path.join(base_path, 'reddish/reddish.resampled33-33-33.10000.csv')
#train_x, train_y = load_data(train_path, fields=['comment_id', 'text', 'abusive'], transform={"NOT":0, "IMP":1, "EXP":2})

# use this for the distant supervision data in addition to AbusEval
train_path = os.path.join(base_path, 'reddish/abuseval+reddish.resampled33-33-33.10000.csv')
train_x, train_y = load_data(train_path, fields=['id', 'text', 'abuse'], transform={"NOTABU":0, "IMP":1, "EXP":2})

test_path = os.path.join(base_path, 'abuseval/test.csv')
test_x, test_y = load_data(test_path, delimiter="\t", fields=['id', 'text', 'abuse'], transform={"NOTABU":0, "IMP":1, "EXP":2})



In [0]:
# @title Build Custom Model
import keras
from keras_radam import RAdam

inputs = model.inputs[:2]
dense = model.get_layer('NSP-Dense').output

# change units={2|3} depending on the classification task
outputs = keras.layers.Dense(units=3, activation='softmax')(dense)

model = keras.models.Model(inputs, outputs)
model.compile(
    RAdam(lr=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)


In [0]:
# @title Fit

model.fit(
    train_x,
    train_y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
)

Epoch 1/5
  344/23112 [..............................] - ETA: 26:01:27 - loss: 1.1613 - sparse_categorical_accuracy: 0.2616

In [0]:
# @title Predict
predicts = model.predict(test_x, verbose=True).argmax(axis=-1)

In [0]:
# @title Accuracy
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support

accuracy = (np.sum(test_y == predicts) / test_y.shape[0])
print (confusion_matrix(test_y, predicts))
report = classification_report(test_y, predicts, output_dict=True)
df = pd.DataFrame(report)
print(df)

tp = 0
fp = 0
tn = 0
fn = 0
for c in [0,1]:
    for g, p in zip(test_y, predicts):
        if g == p: 
            if p==c:
                tp+=1
            else:
                tn+=1
        else:
            if p==c: 
                fp+=1
            else: 
                fn+=1
microp = tp/(tp+fp)
micror = tp/(tp+fn)
microf1 = (2*microp*micror)/(microp+micror)

print ("\t".join(["{0:.3f}".format(v) for v in [
       df['0']['precision'],
       df['0']['recall'],
       df['0']['f1-score'],
       df['1']['precision'],
       df['1']['recall'],
       df['1']['f1-score'],
       df['2']['precision'],
       df['2']['recall'],
       df['2']['f1-score'],
       df['macro avg']['precision'],
       df['macro avg']['recall'],
       df['macro avg']['f1-score'],
       microp, 
       micror, 
       microf1,
       accuracy]]).replace(".", ","))



       